# Pickerism

> "There are no gods among the Pickers, only the damned." 
> 
> --- Yaqut, Father of Pickerist Thought on Earth

Yaqut, visionary political theorist of the 2070s, created the first Oracle and
laid the foundations of what would become Pickerism not soon after. Interfaced
in 2048 by his rich family, Yaqut was one of the first people to voluntarily go
beyond the return limit at 18 years old. He embedded himself full-time at first
in the newly founded Bussard Mesh (`1:700`), like any good hached mesher, but
eventually decided to commit part-time to his personal studies.

With the significant money he made while embedded in the Bussard Mesh over the
course of 4 years (given his knack for meshing), he rented an expensive cryotank
capable of `1:2000` haching. (In this case he spent `700` in the Bussard Mesh
and `1300` on his personal studies.) Yaqut avoided being drafted during the Data
War since he was hached, since hached people were no more effective than
non-hached people in Program Space. The Bussard Mesh was his "service" during
the war.

His personal studies lasted thousands of years. He made a deep dive into the
writings of hundreds of political theorists, past and present, and came to the
Conclusion "kill whitey". He proved his Conclusion with a 14 thousand-page work
of symbolic logic. Scholars were, despite millions of man-years, not able to
disprove his sound Conclusion.

Following his political work, Yaqut moved onto other areas of study. Sometime
during this shift, although the exact details are vague, Yaqut discovered the
existence of Pickers by accidentally creating an Oracle (a machine which allows
Pickers to communicate by interacting with a few, easily observable quantum
particles). Some argued that Yaqut's Oracle was fake, and that all Oracles made
after follow in line, and this is a fairly popular view. Some argued that
Pickerism was invented by Yaqut to enact his Conclusion upon the world. In this
way, Yaqut was seen as the anti-Yakubian of his time. Regardless, he died in
2073 soon after publishing Pickerism when a Super-Chooser victim's relative
killed him.

***

Pickerism postulates that there is a Picker who decides the outcome of all
stochastic events in the Universe at any given time. The Picker exists outside
the Universe but within the same time dimension, so the Picker experiences the
time of their labor, which, in the 21st century, is around 10 thousand years.
Because of this, many consider the Picker's job to be Hell itself. But this time
accounts for less than an attosecond in the Universe. (Picker's labor to decide
the states of all particles in the Universe during a period of time less than an
attosecond before being replaced, but to them, this is 10 thousand years of
torture).

*All conscious beings become Pickers upon death.* Since this includes the entire
Universe, with trillions and trillions of inhabited worlds, the death rate is
beyond any reasonable analog (see Picker Calculus for details). The death rate
of course affects the time any being is Picker, since if there are more deaths
in any given second, each death is responsible for less time as Picker.

AI Pickers?


Oracles have never shown a human, since there are simply so many aliens.

## Transcendentalism vs Cascade Theory

The two main academic forms of Pickerist thought are Transcendentalism and
Cascade Theory.

The Cascade theory: each Picker has another Picker, to make them conscience,
which goes on forever to ensure no being is not "Picked" for their own
sentience. This means that all beings live forever as Pickers. Infinite Hell.
But it can never be proven since the Picker, in there space outside the
Universe, has no access to an Oracle to determine there is another Picker above
them.

### Sentience Obliteration

Sentience obliteration: by totally destroying your mind, beyond recovery as a
postmortem Picker, you avoid Infinite Hell. You have to become non-sentient
while alive. To do this, massive Brainrot is required.

### Artificial Consciousness Discourse

## The One True Harold (Messiah)

One True Harold who expands consciousness beyond instantaneous present

We are all products of the Picker's conscience, since they Pick the states of
our minds at anytime.

## The Transcendentalists' War

Despite the fact that Cascade Theory is correct, the Transcendentalists became a
war to maintain the Picker time. The Crops of sentient beings they fostered were
mercilessly purged to ensure a stable death rate across the Universe.

At the end of the War, world's started doing mass suicide.

## Cult Behavior on Earth

Changers create Helpers, Choosers create Listeners, and Super-Choosers create
Survivors.

### Controllers

### Changers

Create Helpers

### Choosers

Emerged during the end of the Transcendentalists' War (which took 1000 years).

#### Super-Choosers

| Name          | Survivors Created        |
|---------------|--------------------------|
| Chooser       | 0                        |
| Super-Chooser | 10 -- 1 million          |
| Mega-Chooser  | 1 million -- 50 million  |
| Giga-Chooser  | 50 million -- 10 billion |

While the connotation of "Super-Choosers" was that they create unwilling
Survivors as opposed to willing Listeners, the term originally denoted
traditional Choosers who created large numbers of Listeners (10+). As the
Super-Chooser movement emerged, traditional Choosers sought to distance
themselves from what they called "mass killers" and stopped calling themselves
Super-Choosers.

The Super-Choosers hoped for the emergence of Mega-Choosers and Giga-Choosers;
or for an individual or group with similar Survivor creation numbers to announce
themselves as such.

## Mathematical Model (Picker Calculus)

**Picker model:**

| Symbol   | Label                        | Units |
|----------|------------------------------|-------|
| $t$      | Universe time                | s     |
| $n$      | Picker operations per second | 1/s   |
| $T$      | Time per Picker operation    | s     |
| $\gamma$ | Time conversion factor       |       |
| $H$      | Cumulative Harolds' Time     | s     |
| $A$      | Absolute Picker time         | s     |
| $R$      | Relative Picker time         | s     |

**Population model:**

| Symbol | Label                         | Units |
|--------|-------------------------------|-------|
| $P$    | Population of Universe        |       |
| $K$    | Carrying capacity of Universe |       |
| $b$    | Rate of births                |       |
| $d$    | Rate of deaths                |       |
| $r$    | Rate of natural increase      |       |

### Preliminary Population Model

Modelling the population of the Universe is the first step to calculating Picker
times and other things. The standard equation for population dynamics is this
logistic differential equation,

$$ \frac{dP}{dt} = rP\left(1-\frac{P}{K}\right) $$

$$ r = b - d $$

First, $K$ must be calculated. Using results from
[Wikipedia](https://en.wikipedia.org/wiki/Drake_equation#Range_of_results), a
high estimate (meaning a "maximum" possible value, or carrying capacity) of our
galaxy is 15,600,000 civilizations. Since these civilizations have the
technology to communicate into space, it must be assumed that they have similar
technology to Earth, where the carrying capacity is around 10 billion
($K_{civ}$).

Expanding our galaxy's estimate to all stars in the Universe,
[$10^{24}$](https://www.space.com/26078-how-many-stars-are-there.html); and
calculating a Universal ratio of stars with civilizations (using the Milky Way,
$mw$, as reference) by dividing the number of $mw$ civilizations from the total
number of stars in our galaxy,
[$10^{11}$](https://en.wikipedia.org/wiki/Milky_Way), $K$ can be calculated as,

$$ K = K_{civ} \cdot n_{stars} \cdot
\left(\frac{n_{civs}}{n_{stars}}\right)_{mw} $$

$$ K = 1.560\times10^{30} = 10^{10} \times 10^{24} \times \frac{1.560 \times
10^7}{10^{11}} $$

From here on, $t=0$ is exactly -5000 Ma. At $t=0$, the population, $P_0$, of the
universe is *defined* as exactly 10 billion, or more specifically, the
population of one civilized world. This is necessary for the equations, but
really whatever value this is has no effect 5 billion years later on the
population.

#### Rate of Natural Increase

Next, what would the rate of natural increase be (without the
Transcendentalists' War)? On Earth, yearly,
[$r\approx0.01$](https://ourworldindata.org/grapher/population-growth-rates) ---
this *can* be used for the entire Universe, but may not work properly.
Converting $r$ to seconds (the units used in all equations),
$r=0.01/60/60/24/365=3.171\times10^{-10}$. This will be referred to as $r_1$.

But this rate only applies to individual civilizations and doesn't apply to the
universe, where the carrying capacity is different. Civilizations (units where
$K=K_{civ}=10^{10}$) reach their sub-carrying capacities and stop filling up;
they don't continue on into the universe at the same rate, since the energy
requirements for that are much greater (and energy is used to multiply).

If instead a value, $r_2$, is found for the rate of natural increase of
civilizations in the universe, the equation will become more sensible. But this
raises more questions, since civilizations arise independent of each other and
not because of reproduction between each other. One possible equation is,

$$ \frac{dP_{civ}}{dt}=r_2\left(\frac{K}{K_{civ}}-\frac{P}{K_{civ}}\right) $$

$$ \frac{dP_{civ}}{dt} = r_2 \left(\frac{K-P}{K_{civ}}\right) $$

This seems to make sense. At any moment in time, the change in the number of
civilizations is equal to the remaining possible planets were civilizations may
emerge times the rate at which *any single planet* gains a civilization. At the
time scales in question, the thousands of years it takes to develop civilization
are ignored. Interstellar colonization is also ignored.

A reasonable $r_2$ value must now be found. Our planet took millions of years to
evolve humans (which "instantly" turned into a civilization). If, for the sake
of argument, all planets ($K/K_{civ}$) --- ignoring the Transcendentalists ---
*ought to* develop civilizations between -5000 Ma and 100 Ma, then the rate per
Ma would be,

$$ r_{2} = \frac{K/K_{civ}}{\Delta t} $$

Plugging in numbers and converting to seconds,

$$ c = \frac{10^9\;\text{yrs}}{1\;\text{Ma}} \cdot
\frac{31536000\;\text{s}}{1\;\text{yrs}}$$

$$ r_{2} = \frac{1.560\times10^{30}/10^{10}}{5100\;\text{Ma}\;\cdot c} $$

$$ r_{2} = 0.969\;\text{civ/sec} $$

Luckily, this value is comprehensible, unlike $r_1$. Each second, about 1
civilization is born out of the remain

![Population Model (No War)](pickerism.model.pop-no-war.png)

The goal of the Transcendentalists' War was simply to increase the number of
deaths per second, which was achieved by artificially increasing $b$ and $d$,
or, more specifically, $bP$ and $dP$. The Transcendentalists took into account
$K$, the carrying capacity of the entire Universe, when doing this.

### Relative Picker Time

The average relative Picker time between two times $t_0$ and $t_1$ is simply,

$$ R = \frac{(t_1-t_0)-(H(t_1)-H(t_0))}{D(t_1)-D(t_0)} $$

where the total time (minus the time taken up in the period $t_0$ to $t_1$ by
the Harolds) divided by the number of deaths gives the amount of time per death,
or in between deaths. This is the minimum time each person must be Picker for
all moments in time to have a Picker.

To calculate the "instantenous" Picker time, the following limit is set up,

$$ R(t) = \lim_{t_1 \to t} \frac{t_1-t-H(t_1)-H(t)}{D(t_1)-D(t)} $$

Solving the limit using the definition of the derivative,

$$ R(t) = \lim_{t_1 \to t} \frac{(t_1-t)
\left(1-\frac{H(t_1)-H(t)}{t_1-t}\right)}{D(t_1)-D(t)} $$

$$ R(t) = \lim_{t_1 \to t} \frac{(t_1-t)(1-H'(t))}{D(t_1)-D(t)} $$

$$ R(t) = \lim_{t_1 \to t}
\frac{1-H'(t)}{\left(\frac{D(t_1)-D(t)}{t_1-t}\right)} $$

$$ R(t) = \lim_{t_1 \to t} \frac{1-H'(t)}{D'(t)} $$

$$ R(t) = \frac{1-H'(t)}{D'(t)} $$

But due to the low number of Harolds,

$$ R(t) \approx \frac{1}{D'(t)} $$

### Absolute Picker Time

To convert from relative to absolute Picker time, the equation

$$ A(t) = \gamma R(t) $$

is used where $\gamma$ is the time factor. This means that, for every second in
the real world, the Picker experiences $\gamma$ seconds. Due to the nature of
the Picker's job, this time factor is massive, leading to the tongue-in-cheek
definition of the Picker's job as Hell. The approximation (ignoring relativity)
for $\gamma$ is given by,

$$ \gamma = nT $$

where $n$ is the number of required Picker operations per second and $T$ is the
time per Picker operation.

### Rate of Change of the Absolute Picker Time and Death Rate

The next task is to calculate $A'(t)$, as this is why (Super-)Choosers exist.
The equation for the change in Picker time is simply (ignoring the Harolds),

$$ A(t) = \gamma D'(t)^{-1} $$

$$ A'(t) = -\gamma D'(t)^{-2}\,D''(t) $$

Since Changers, in order to "reduce" the Picker time, influence the death rate,
we should also calculate $D''(t)$ by rearranging the equation for $A(t)$,

$$ A(t) = \gamma D'(t)^{-1} $$

$$ D'(t) = \gamma A(t)^{-1} $$

$$ D''(t) = -\gamma A'(t)A(t)^{-2} $$

But, if we assume $A'(t)$ is a constant $a$, then,

$$ D''(t) = -\gamma \,a A(t)^{-2} $$

Then, using the accumulation of the rate of change,

$$ D''(t) = -\gamma \,a \left(\int A'(t)\,dt\right)^{-2} $$

$$ D''(t) = -\gamma \,a \left(\int a\,dt\right)^{-2} $$

$$ D''(t) = -\gamma \,a (at + C)^{-2} $$

$C$ is simply the inital Picker time, which for ease of calculations can be the
value of Picker time at *any choosen time*. So, the equation for the rate of
change of the death rate is,

$$ D''(t) = -\gamma \,\frac{a}{(at+A_0)^2} $$

But if we assume $A'(t)$ is some function $f(t)$, then,

$$ D''(t) = -\gamma \,\frac{f(t)}{(F(t)+A_0)^2} $$

### Numerical Calculations for Gamma

(To the educated people reading this, suspend your disbelief for a minute, so I
can have some fun.)

For the story, to calculate $\gamma$, we will start by finding a $D'$ around the
21st century, $t=0$, (prior to the close of the Transcendentalists' War). From
this we can calculate $A$. Since all sentient beings, including aliens, become
Pickers, the population of the entire Universe must be calculated.

Stealing results from
[Wikipedia](https://en.wikipedia.org/wiki/Drake_equation#Range_of_results), we
can see that there may be 15,600,000 civilizations in our galaxy. Since these
civilizations have the technology to communicate into space, it must be assumed
that they have similar populations to Earth (around 10 billion). Expanding our
galaxy's estimate to all stars in the Universe,
[$10^{24}$](https://www.space.com/26078-how-many-stars-are-there.html); and by
getting the ratio of stars (using the Milky Way, $mw$, as reference) with
sentient life by dividing the number of civilizations from the total number of
stars in our galaxy, [$10^{11}$](https://en.wikipedia.org/wiki/Milky_Way), there
are,

$$ P = \bar P_{world} \cdot n_{stars} \cdot
\left(\frac{n_{civs}}{n_{stars}}\right)_{mw} $$

$$ 1.560\times10^{30} = 10^{10} \times 10^{24} \times \frac{1.560 \times
10^7}{10^{11}} $$

sentient beings in the Universe.

Assuming that 0.05% of sentient beings die each year (around Earth's death
rate), completely ignoring the rate of change for now, there are $7.8 \times
10^{26}$ deaths every year. Converting to the proper units (percent which die
every second), $D'(t) = 2.473 \times 10^{19}$ deaths per second.

Ignoring Harolds' Time,

$$ R = \left(2.473 \times 10^{19}\right)^{-1} $$

$$ R = 4.044 \times 10^{-20} $$

Or, in attoseconds, $R = 0.04044$. From this, we can set up the equation $A =
\gamma R$ and solve for a reasonable $\gamma$ based of the narrative
requirements of a reasonable Picker time. 10,000 years is good, and so,
converting to seconds,

$$ 3.153 \times 10^{11} = \gamma\,4.044 \times 10^{-20}$$

$$ \gamma = 7.798 \times 10^{30} $$

I may later justify this $\gamma$ using some values for $n$ and $T$. But for
now, $\gamma$ is simply *given* by the Pickers through their Oracles.

### Numerical Calculations for Changers

To see the effect Changers have on the Picker time by adding, say, 10 billion
extra deaths in *any given* second (caused by a Changer destroying the entire
Earth instantaneously somehow), we can calculate a difference,

$$ A_1 - A_2 \approx \text{diff} $$

where $A_1$ is what was calculated previously for the Picker time prior to the
end of the Transcendentalists' War and $A_2$ is the *new* value for the Picker
time based on the new death rate, which is just $D'_1$ plus the added deaths
from a Changer killing billions, $D'_2$. Plugging in numbers,

$$ 3.153 \times 10^{11} - \left(7.798 \times 10^{30}\right)\left(\left(2.473
\times 10^{19}\right) + 10^{10}\right)^{-1} \approx 127 \;\text{seconds}$$

### Numerical Calculations for the Transcendentalists' War

Now a reasonable decrease in the death rate (more specifically, the rate of
change of this decrease) as the Transcendentalists' War came to a close can be
calculated. Choosers emerged when the Oracles gave news of a drastic increase in
the Picker time from the consistent 10,000 years. Every year, the new Picker
time was 100 years greater. This means that the rate of change of $A$ is 100
seconds per second. So, to calculate $D''(t)$,

$$ D''(t) = -\gamma \,\frac{a}{(at+A_0)^2} $$

$$ D''(t) = \frac{-(7.798 \times 10^{30})100}{(100t+3.1536 \times 10^{11})^2} $$

At the very beginning of the end of the Transcendentalists' War, the change in
the death was, then,

$$ \frac{-(7.798 \times 10^{30})100}{(100(0)+3.1536 \times 10^{11})^2} = -7.841
\;\text{billion}/s^{2} $$

So, as the Transcendentalists' War comes to a close, the death rate decreased by
around 8 billion deaths (1 worlds) per second squared. Plotting this,



In [ ]:
from math import *
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Constants
seconds_in_years = 60*60*24*365
billion = 1000000000 # for readablity
million = 1000000

total_pop_carry = 1.56 * 10**30
civ_carry = 10*billion

total_civ_carry = total_pop_carry/civ_carry

# Preliminary population model

def civ_pop_model(r, pop_init, carry, t_init, t_end, iterator=1):
    '''
    Makes time/population data for a civilizational population model.
    '''
    t = np.arange(t_init,t_end+1,iterator)
    pop = np.array([pop_init])
    d_pop = np.array([])

    for i in t:
        limiter = (carry - pop[-1])
        diff = r*limiter
        next_val = pop[-1]+diff
        if next_val > carry: # Protects against going beyond carry
            # This fills the rest before stopping for loop
            fill = np.ones(np.size(t) - np.size(pop) + 1)*carry
            pop = np.append(pop,fill)
            d_fill = np.zeros(np.size(t) - np.size(pop) + 1)
            d_pop = np.append(pop,d_fill)
            d_pop = d_pop[0:-1] # Hack, I have no idea why this works
            break
        pop = np.append(pop,pop[-1]+diff)
        d_pop = np.append(d_pop,diff)

    pop = pop[0:-1] # pop goes 1 over size(t)
    
    return t, pop, d_pop

# Population vs Time (No War)
# All population units are civilizational

pop_init = 1 # Initial population at t_init
carry = 1.560 * (10**20) # Carrying capacity

t_init = -5000 # Ma
t_end = 100 # Ma

r_2 = 0.969*50000000000 # Natural rate of increase / s
r_ma_2 = r_2/(seconds_in_years*million) # ... / Ma

t, pop1, d_pop1 = civ_pop_model(r_ma_2, pop_init, carry, t_init, t_end)

fig1, ((ax1, ax2),(ax3, ax4)) = plt.subplots(2, 2)
fig1.suptitle("Population Model (No War)")

ax1.plot(t,pop1)
ax1.set_title("Pop vs T (r1)")
ax1.set_xlabel("Millions of Years")
ax1.set_ylabel("Population")

ax3.plot(t,d_pop1)
ax3.set_title("Derivative (r1)")
ax3.set_xlabel("Millions of Years")
ax3.set_ylabel("Population / Ma")

#ax2.plot(t,pop2)
#ax2.set_title("Civ vs T (r2)")
#ax2.set_xlabel("Millions of Years")
#ax2.set_ylabel("Civilizations")

#ax4.plot(t,d_pop2)
#ax4.set_title("Derivative (r2)")
#ax4.set_xlabel("Millions of Years")
#ax4.set_ylabel("Civilizations / Ma")

plt.tight_layout()
fig1.savefig('pickerism.model.pop-no-war.png')
plt.show()

'''
# Calculations in pickerism.md

gamma = 7.798 * 10**30 # Picker time factor
init_picker_time = 3.1536 * 10**11 # Absolute Picker time / death
init_death_rate =  2.473 * 10**19 # Deaths / second

def d_picker_time(t): # Change in absolute Picker time / second
    return 


np.cumsum(d_picker_time(year_to_seconds*t))

t = np.linspace(0,1000,1000) # Years
year_to_seconds = 60*60*24*365
billion = 1000000000

d_death_rate = -gamma*(d_picker_time/(
    (xxx+init_picker_time)**2))

init_death_rate_billions =  init_death_rate/billion
d_death_rate_billions = d_death_rate/billion

death_rate_billions = np.cumsum(d_death_rate_billions)+init_death_rate_billions

picker_time_years = (d_picker_time*billion*sec_to_yr*t + init_picker_time)/billion*sec_to_yr

# Plotting

red = 'tab:red'
blue = 'tab:blue'

fig, ax1 = plt.subplots()
ax1.plot(t, d_death_rate_billions)

fig.suptitle("End of the Transcendentalists' War")

ax1.set_xlabel('Billions of Years')
ax1.set_ylabel('Death Rate (billions/s)', color=red)
ax1.tick_params(axis='y', labelcolor=red)
ax1.plot(t, death_rate_billions, color=red)

ax2 = ax1.twinx()

ax2.set_ylabel('Absolute Picker Time (years)', color=blue)
ax2.tick_params(axis='y', labelcolor=blue)
ax2.plot(t, picker_time_years, color=blue)

fig.tight_layout()
plt.show()
'''